In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [20]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2022-01'
post_data['gender'] = 'all'
post_data['page'] = None
actors_info_list = list()

for i in range(1,11):
    post_data['page'] = i
    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.text, 'html.parser')
    actors = soup.select('li.people_li div.name')
    strong_scores = soup.select('strong')
    ranks = soup.select('span.grade')

    movies = [list(map(lambda x : x.text, soup.select('ul.mov_list')[i].select('li > a > span'))) for i in range(len(actors))]

    for actor, score, movie, rank in zip(actors, strong_scores, movies,ranks):
        actor_link = 'http://www.cine21.com' + actor.select_one('a')['href'] # 태그 내 속성 접근
        response_actor = requests.get(actor_link) # 배우 상세 페이지 이동
        soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        actor_info_dict = dict()
        actor_info_dict['이름'] = re.sub('\(\w*\)', '', actor.text)
        actor_info_dict['랭킹'] = rank.text
        actor_info_dict['흥행점수'] = score.text
        actor_info_dict['출현영화'] = movie

        for actor_detail in actor_details:
            actor_detail_key = actor_detail.select_one('span.tit').text # 상세정보 key값
            actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail)) # span 태그 제거
            actor_detail_value = re.sub('<.*?>', '', actor_detail_value) # 상세정보 value값
            actor_info_dict[actor_detail_key] = actor_detail_value

        actors_info_list.append(actor_info_dict)

In [21]:
actors_info_list

[{'이름': '강하늘',
  '랭킹': '1',
  '흥행점수': '79,392',
  '출현영화': ['해적: 도깨비 깃발', '해피 뉴 이어'],
  '다른 이름': '김하늘',
  '직업': '배우',
  '생년월일': '1990-02-21',
  '성별': '남',
  '홈페이지': '\nhttp://weibo.com/galpos3?is_hot=1\n',
  '신장/체중': '181cm, 70kg',
  '학교': '중앙대학교 연극학과'},
 {'이름': '한효주',
  '랭킹': '2',
  '흥행점수': '67,602',
  '출현영화': ['해적: 도깨비 깃발'],
  '직업': '배우',
  '생년월일': '1987-02-22',
  '성별': '여',
  '홈페이지': '\nhttps://www.facebook.com/hhj.official\n',
  '신장/체중': '170cm',
  '학교': '동국대학교 연극영화',
  '취미': '영화감상'},
 {'이름': '이광수',
  '랭킹': '3',
  '흥행점수': '59,677',
  '출현영화': ['해적: 도깨비 깃발', '해피 뉴 이어'],
  '직업': '배우',
  '생년월일': '1985-07-14',
  '성별': '남',
  '홈페이지': '\nhttps://twitter.com/masijacoke85\nhttps://www.instagram.com/masijacoke850714/\n',
  '신장/체중': '190cm',
  '소속사': '킹콩엔터테인먼트'},
 {'이름': '권상우',
  '랭킹': '4',
  '흥행점수': '48,287',
  '출현영화': ['해적: 도깨비 깃발'],
  '다른 이름': 'Kwon Sang Woo',
  '직업': '배우',
  '생년월일': '1976-08-05',
  '성별': '남',
  '신장/체중': '183cm, 72kg',
  '학교': '한남대학교 미술교육학 학사',
  '취미': '수영, 헬스, 복싱',
  '특기':